In [1]:
# Credits: https://github.com/SullyChen/Autopilot-TensorFlow
# Research paper: End to End Learning for Self-Driving Cars by Nvidia. [https://arxiv.org/pdf/1604.07316.pdf]

# NVidia dataset: 72 hrs of video => 72*60*60*30 = 7,776,000 images
# Nvidia blog: https://devblogs.nvidia.com/deep-learning-self-driving-cars/


# Our Dataset: https://github.com/SullyChen/Autopilot-TensorFlow [https://drive.google.com/file/d/0B-KJCaaF7elleG1RbzVPZWV4Tlk/view]
# Size: 25 minutes = 25*60*30 = 45,000 images ~ 2.3 GB


# If you want to try on a slightly large dataset: 70 minutes of data ~ 223GB
# Refer: https://medium.com/udacity/open-sourcing-223gb-of-mountain-view-driving-data-f6b5593fbfa5
# Format: Image, latitude, longitude, gear, brake, throttle, steering angles and speed

import scipy.misc
import tensorflow as tf
import scipy
import random
import numpy as np
from __future__ import division
import datetime

import os
import numpy as np
import random

from scipy import pi
from itertools import islice
import model
import cv2
from subprocess import call


# Additional Installations:
# pip3 install h5py


# AWS: https://aws.amazon.com/blogs/machine-learning/get-started-with-deep-learning-using-the-aws-deep-learning-ami/

# Youtube:https://www.youtube.com/watch?v=qhUvQiKec2U
# Further reading and extensions: https://medium.com/udacity/teaching-a-machine-to-steer-a-car-d73217f2492c
# More data: https://medium.com/udacity/open-sourcing-223gb-of-mountain-view-driving-data-f6b5593fbfa5

## Data analysis from steering angles

In [2]:
# read images and steering angles from driving_dataset folder

from __future__ import division

import os
import numpy as np
import random

from scipy import pi
from itertools import islice



DATA_FOLDER = './driving_dataset' # change this to your folder
TRAIN_FILE = os.path.join(DATA_FOLDER, 'data.txt')
LIMIT = None

split =0.7
X = []
y = []
with open(TRAIN_FILE) as fp:
    for line in islice(fp, LIMIT):
        path, angle = line.strip().split()
        full_path = os.path.join(DATA_FOLDER, path)
        X.append(full_path)
        
        # converting angle from degrees to radians
        y.append(float(angle) * pi / 180 )


y = np.array(y)
print("Completed processing data.txt")

split_index = int(len(y)*split)

train_y = y[:split_index]
test_y = y[split_index:]


Completed processing data.txt


In [3]:
print(np.shape(train_y))
print(np.shape(test_y))

(31784,)
(13622,)


In [4]:
import numpy
import matplotlib.pyplot as plt 

plt.figure(figsize=(12,8))
plt.hist(train_y, bins=50, normed=1, color='blue', histtype ='step');
plt.hist(test_y, bins=50, normed=1, color='red', histtype ='step');
plt.show()

C:\Anaconda3\lib\site-packages\matplotlib\axes\_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


<Figure size 1200x800 with 1 Axes>

### Baseline model

In [5]:
#Model 0: Base line Model: y_test_pred = mean(y_train_i) 
train_mean_y = np.mean(train_y)

print('Test_MSE(MEAN):%f' % np.mean(np.square(test_y-train_mean_y)) )

print('Test_MSE(ZERO):%f' % np.mean(np.square(test_y-0.0)) )


Test_MSE(MEAN):0.241561
Test_MSE(ZERO):0.241107


### Driving_data

In [6]:
import scipy.misc
import random

xs = []
ys = []

#points to the end of the last batch
train_batch_pointer = 0
val_batch_pointer = 0

#read data.txt
with open("driving_dataset/data.txt") as f:
    for line in f:
        xs.append("driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)
split =0.7

train_xs = xs[:int(len(xs) * split)]
train_ys = ys[:int(len(xs) * split)]

val_xs = xs[-int(len(xs) * 0.3):]
val_ys = ys[-int(len(xs) * 0.3):]

num_train_images = len(train_xs)
num_val_images = len(val_xs)

def LoadTrainBatch(batch_size):
    global train_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:], [66, 200]) / 255.0)
        y_out.append([train_ys[(train_batch_pointer + i) % num_train_images]])
    train_batch_pointer += batch_size
    return x_out, y_out

def LoadValBatch(batch_size):
    global val_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(val_xs[(val_batch_pointer + i) % num_val_images])[-150:], [66, 200]) / 255.0)
        y_out.append([val_ys[(val_batch_pointer + i) % num_val_images]])
    val_batch_pointer += batch_size
    return x_out, y_out


In [7]:
print(np.shape(ys))
print(np.shape(xs))
ys[:10]

(45406,)
(45406,)


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [8]:
scipy.misc.imresize(scipy.misc.imread(train_xs[0])[-150:], [66, 200])

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  """Entry point for launching an IPython kernel.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  """Entry point for launching an IPython kernel.


array([[[180, 162, 166],
        [176, 172, 173],
        [176, 176, 171],
        ...,
        [ 90,  88, 113],
        [106,  93,  99],
        [101, 103,  81]],

       [[191, 188, 192],
        [186, 193, 204],
        [187, 196, 200],
        ...,
        [ 84,  82,  97],
        [ 86,  88,  79],
        [ 86, 101,  74]],

       [[208, 201, 223],
        [199, 212, 230],
        [201, 212, 226],
        ...,
        [128, 124, 115],
        [128, 126, 117],
        [132, 126, 119]],

       ...,

       [[ 54,  43,  55],
        [ 59,  43,  56],
        [ 55,  41,  53],
        ...,
        [ 23,  24,  25],
        [ 24,  25,  27],
        [ 25,  26,  29]],

       [[ 56,  36,  58],
        [ 53,  35,  63],
        [ 51,  39,  54],
        ...,
        [ 23,  25,  22],
        [ 23,  26,  23],
        [ 24,  27,  25]],

       [[ 68,  37,  44],
        [ 53,  41,  49],
        [ 49,  49,  37],
        ...,
        [ 28,  25,  26],
        [ 26,  23,  25],
        [ 24,  22,  24]]

### Model with Linear Activation unit

In [9]:
import tensorflow as tf
import scipy

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W, stride):
  return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='VALID')

x = tf.placeholder(tf.float32, shape=[None, 66, 200, 3])
y_ = tf.placeholder(tf.float32, shape=[None, 1])

x_image = x

#first convolutional layer
W_conv1 = weight_variable([5, 5, 3, 24])
b_conv1 = bias_variable([24])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 2) + b_conv1)

#second convolutional layer
W_conv2 = weight_variable([5, 5, 24, 36])
b_conv2 = bias_variable([36])

h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 2) + b_conv2)

#third convolutional layer
W_conv3 = weight_variable([5, 5, 36, 48])
b_conv3 = bias_variable([48])

h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 2) + b_conv3)

#fourth convolutional layer
W_conv4 = weight_variable([3, 3, 48, 64])
b_conv4 = bias_variable([64])

h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4, 1) + b_conv4)

#fifth convolutional layer
W_conv5 = weight_variable([3, 3, 64, 64])
b_conv5 = bias_variable([64])

h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, 1) + b_conv5)

#FCL 1
W_fc1 = weight_variable([1152, 1164])
b_fc1 = bias_variable([1164])

h_conv5_flat = tf.reshape(h_conv5, [-1, 1152])
h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

keep_prob = 0.5                                # dropout of 50%
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#FCL 2
W_fc2 = weight_variable([1164, 100])
b_fc2 = bias_variable([100])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

#FCL 3
W_fc3 = weight_variable([100, 50])
b_fc3 = bias_variable([50])

h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)

#FCL 3
W_fc4 = weight_variable([50, 10])
b_fc4 = bias_variable([10])

h_fc4 = tf.nn.relu(tf.matmul(h_fc3_drop, W_fc4) + b_fc4)

h_fc4_drop = tf.nn.dropout(h_fc4, keep_prob)

#Output
W_fc5 = weight_variable([10, 1])
b_fc5 = bias_variable([1])

# https://keras.io/activations/
# https://www.tensorflow.org/api_docs/python/tf/keras/activations/linear

y = tf.multiply(tf.keras.activations.linear(tf.matmul(h_fc4_drop, W_fc5) + b_fc5), 2) 


### Training the data
#### Linear activation unit 

In [10]:
import os
import tensorflow as tf
from tensorflow.core.protobuf import saver_pb2
import driving_data
import model

startTime = datetime.datetime.now()
print("Current Time = ",startTime)

LOGDIR = './save_assignment'

sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(model.y_, model.y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)
sess.run(tf.initialize_all_variables())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op =  tf.summary.merge_all()

saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V1)

# op to write logs to Tensorboard
logs_path = './logs_assignment'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 7
batch_size = 100

# train over the dataset about 30 times
for epoch in range(epochs):
  for i in range(int(driving_data.num_images/batch_size)):
    xs, ys = driving_data.LoadTrainBatch(batch_size)
    train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.8})
    if i % 10 == 0:
      xs, ys = driving_data.LoadValBatch(batch_size)
      loss_value = loss.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
      print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

    # write logs at every iteration
    summary = merged_summary_op.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
    summary_writer.add_summary(summary, epoch * driving_data.num_images/batch_size + i)

    if i % batch_size == 0:
      if not os.path.exists(LOGDIR):
        os.makedirs(LOGDIR)
      checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
      filename = saver.save(sess, checkpoint_path)
  print("Model saved in file: %s" % filename)

print("Run the command line:\n" \
          "--> tensorboard --logdir=./logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")

print("Time taken for creation of dataframe is {}".format(datetime.datetime.now() - startTime))


Current Time =  2019-05-19 10:43:52.623245
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch: 0, Step: 0, Loss: 12.6059
Epoch: 0, Step: 10, Loss: 12.3919
Epoch: 0, Step: 20, Loss: 12.0305
Epoch: 0, Step: 30, Loss: 11.8632
Epoch: 0, Step: 40, Loss: 11.6801
Epoch: 0, Step: 50, Loss: 11.5964
Epoch: 0, Step: 60, Loss: 11.6064
Epoch: 0, Step: 70, Loss: 12.2779
Epoch: 0, Step: 80, Loss: 12.8835
Epoch: 0, Step: 90, Loss: 11.1854
Epoch: 0, Step: 100, Loss: 10.8283
Epoch: 0, Step: 110, Loss: 10.6243
Epoch: 0, Step: 120, Loss: 10.4824
Epoch: 0, Step: 130, Loss: 10.355
Epoch: 0, Step: 140, Loss: 10.2374
Epoch: 0, Step: 150, Loss: 10.1253
Epoch: 0, Step: 160, Loss: 9.97964
Epoch: 0, Step: 170, Loss: 9.81414
Epoch: 0, Step: 180, Loss: 9.78346
Epoch: 0, Step: 190, Loss: 9.92339
Epoch: 0, Step: 200, Loss: 9.4887
Epoch: 0, Step: 210, Loss: 9.45655
Epoch: 0, Step: 220, Loss: 9.19439
Epoch: 0, Step: 230, Loss: 9.07707
Epoch: 0, Step: 240, Loss: 8.961
Epoch: 0, Step: 250, Lo

Epoch: 2, Step: 230, Loss: 4.13975
Epoch: 2, Step: 240, Loss: 4.10671
Epoch: 2, Step: 250, Loss: 4.18013
Epoch: 2, Step: 260, Loss: 5.02065
Epoch: 2, Step: 270, Loss: 5.21621
Epoch: 2, Step: 280, Loss: 3.99393
Epoch: 2, Step: 290, Loss: 4.00017
Epoch: 2, Step: 300, Loss: 3.85876
Epoch: 2, Step: 310, Loss: 3.83962
Epoch: 2, Step: 320, Loss: 3.79318
Epoch: 2, Step: 330, Loss: 3.7792
Epoch: 2, Step: 340, Loss: 3.77235
Epoch: 2, Step: 350, Loss: 3.74029
Epoch: 2, Step: 360, Loss: 3.66806
Epoch: 2, Step: 370, Loss: 3.70614
Epoch: 2, Step: 380, Loss: 3.97548
Epoch: 2, Step: 390, Loss: 3.70673
Epoch: 2, Step: 400, Loss: 3.59167
Epoch: 2, Step: 410, Loss: 3.49881
Epoch: 2, Step: 420, Loss: 3.47023
Epoch: 2, Step: 430, Loss: 3.4432
Epoch: 2, Step: 440, Loss: 3.41426
Epoch: 2, Step: 450, Loss: 3.45995
Epoch: 2, Step: 460, Loss: 8.96706
Epoch: 2, Step: 470, Loss: 3.55604
Epoch: 2, Step: 480, Loss: 3.31091
Epoch: 2, Step: 490, Loss: 3.27897
Epoch: 2, Step: 500, Loss: 3.26287
Epoch: 2, Step: 510, L

Epoch: 4, Step: 550, Loss: 2.03336
Epoch: 4, Step: 560, Loss: 2.14145
Epoch: 4, Step: 570, Loss: 2.4366
Epoch: 4, Step: 580, Loss: 2.25561
Epoch: 4, Step: 590, Loss: 1.98981
Epoch: 4, Step: 600, Loss: 1.96962
Epoch: 4, Step: 610, Loss: 1.96087
Epoch: 4, Step: 620, Loss: 1.95144
Epoch: 4, Step: 630, Loss: 1.94329
Epoch: 4, Step: 640, Loss: 2.40769
Epoch: 4, Step: 650, Loss: 6.99612
Epoch: 4, Step: 660, Loss: 1.9357
Epoch: 4, Step: 670, Loss: 1.92262
Epoch: 4, Step: 680, Loss: 1.92028
Epoch: 4, Step: 690, Loss: 1.90436
Epoch: 4, Step: 700, Loss: 1.91476
Epoch: 4, Step: 710, Loss: 1.92421
Epoch: 4, Step: 720, Loss: 1.91356
Epoch: 4, Step: 730, Loss: 1.87435
Epoch: 4, Step: 740, Loss: 1.89343
Epoch: 4, Step: 750, Loss: 1.92913
Epoch: 4, Step: 760, Loss: 2.01674
Epoch: 4, Step: 770, Loss: 1.85053
Epoch: 4, Step: 780, Loss: 2.06811
Epoch: 4, Step: 790, Loss: 1.8384
Epoch: 4, Step: 800, Loss: 1.79766
Epoch: 4, Step: 810, Loss: 1.79781
Epoch: 4, Step: 820, Loss: 1.79103
Epoch: 4, Step: 830, Lo

Epoch: 6, Step: 870, Loss: 1.35661
Epoch: 6, Step: 880, Loss: 1.34291
Epoch: 6, Step: 890, Loss: 1.33788
Epoch: 6, Step: 900, Loss: 1.3412
Epoch: 6, Step: 910, Loss: 1.34189
Epoch: 6, Step: 920, Loss: 1.3285
Epoch: 6, Step: 930, Loss: 1.33509
Epoch: 6, Step: 940, Loss: 1.33594
Epoch: 6, Step: 950, Loss: 1.34833
Epoch: 6, Step: 960, Loss: 1.30627
Epoch: 6, Step: 970, Loss: 1.30098
Epoch: 6, Step: 980, Loss: 1.29775
Epoch: 6, Step: 990, Loss: 1.28622
Epoch: 6, Step: 1000, Loss: 1.28578
Epoch: 6, Step: 1010, Loss: 1.2789
Epoch: 6, Step: 1020, Loss: 1.28271
Epoch: 6, Step: 1030, Loss: 1.27142
Epoch: 6, Step: 1040, Loss: 1.27487
Epoch: 6, Step: 1050, Loss: 1.2708
Model saved in file: ./save_assignment\model.ckpt
Run the command line:
--> tensorboard --logdir=./logs 
Then open http://0.0.0.0:6006/ into your web browser
Time taken for creation of dataframe is 4:30:05.830714


## Running dataset

In [1]:
#pip3 install opencv-python

import tensorflow as tf
import scipy.misc
import model
import cv2
from subprocess import call
import math

sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, "save_assignment/model.ckpt")

img = cv2.imread('steering_wheel_image.jpg',0)
rows,cols = img.shape

smoothed_angle = 0

#read data.txt
xs = []
ys = []
with open("driving_dataset/data.txt") as f:
    for line in f:
        xs.append("driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)


i = math.ceil(num_images*0.7)
print("Starting frameofvideo:" +str(i))

while(cv2.waitKey(10) != ord('q')):
    full_image = scipy.misc.imread("driving_dataset/" + str(i) + ".jpg", mode="RGB")
    image = scipy.misc.imresize(full_image[-150:], [66, 200]) / 255.0
    degrees = model.y.eval(feed_dict={model.x: [image], model.keep_prob: 1.0})[0][0] * 180.0 / scipy.pi
    #call("clear")
    #print("Predicted Steering angle: " + str(degrees))
    print("Steering angle: " + str(degrees) + " (pred)\t" + str(ys[i]*180/scipy.pi) + " (actual)")
    cv2.imshow("frame", cv2.cvtColor(full_image, cv2.COLOR_RGB2BGR))
    #make smooth angle transitions by turning the steering wheel based on the difference of the current angle
    #and the predicted angle
    smoothed_angle += 0.3 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    cv2.imshow("steering wheel", dst)
    i += 1

cv2.destroyAllWindows()


INFO:tensorflow:Restoring parameters from save_assignment/model.ckpt
Starting frameofvideo:31785


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


Steering angle: -0.019784496704588185 (pred)	-28.34 (actual)
Steering angle: 2.0380441381842376 (pred)	-28.840000000000003 (actual)
Steering angle: 2.1695329636697993 (pred)	-29.75 (actual)
Steering angle: 3.260743195378077 (pred)	-31.06 (actual)
Steering angle: 1.1953149078205543 (pred)	-32.27 (actual)
Steering angle: 0.3455860408985038 (pred)	-33.48 (actual)
Steering angle: 0.9133024801279573 (pred)	-34.39 (actual)
Steering angle: 1.6548363748006722 (pred)	-35.599999999999994 (actual)
Steering angle: 2.0288408851644686 (pred)	-36.5 (actual)
Steering angle: 3.8519829395111866 (pred)	-37.61 (actual)
Steering angle: 3.7734784527383525 (pred)	-38.62 (actual)
Steering angle: 4.126571485184509 (pred)	-39.63 (actual)
Steering angle: 4.17703121522527 (pred)	-39.93000000000001 (actual)
Steering angle: 4.022720593288581 (pred)	-40.03000000000001 (actual)
Steering angle: 4.5552930678761 (pred)	-40.03000000000001 (actual)
Steering angle: 3.9076690449274776 (pred)	-40.03000000000001 (actual)
Stee

Steering angle: 4.0572250014281925 (pred)	-0.61 (actual)
Steering angle: 4.288992102986943 (pred)	-0.61 (actual)
Steering angle: 4.6879233887267 (pred)	-0.61 (actual)
Steering angle: 4.947962350807426 (pred)	-0.61 (actual)
Steering angle: 4.341929911035949 (pred)	-0.61 (actual)
Steering angle: 4.710740062535951 (pred)	-0.61 (actual)
Steering angle: 4.896944252802015 (pred)	-0.5 (actual)
Steering angle: 4.601415200907081 (pred)	-0.3 (actual)
Steering angle: 4.889003731005053 (pred)	0.2 (actual)
Steering angle: 4.792661778327907 (pred)	0.3 (actual)
Steering angle: 4.245723281481735 (pred)	0.4 (actual)
Steering angle: 4.138175549697538 (pred)	0.5 (actual)
Steering angle: 4.063591591508291 (pred)	1.01 (actual)
Steering angle: 4.234138426875747 (pred)	1.6100000000000003 (actual)
Steering angle: 4.785727855659875 (pred)	2.12 (actual)
Steering angle: 4.490333700001048 (pred)	2.52 (actual)
Steering angle: 4.512217199214946 (pred)	3.13 (actual)
Steering angle: 5.356899009907607 (pred)	4.13 (act

Steering angle: 6.634896986169162 (pred)	23.09 (actual)
Steering angle: 6.398341922002232 (pred)	23.09 (actual)
Steering angle: 6.180658670512049 (pred)	23.29 (actual)
Steering angle: 5.680615398733537 (pred)	23.39 (actual)
Steering angle: 5.787964201258158 (pred)	23.29 (actual)
Steering angle: 5.969729622337361 (pred)	22.89 (actual)
Steering angle: 7.518016540189715 (pred)	22.59 (actual)
Steering angle: 7.273678475463557 (pred)	22.59 (actual)
Steering angle: 7.285774740483649 (pred)	22.39 (actual)
Steering angle: 6.946201213725887 (pred)	22.08 (actual)
Steering angle: 6.267655643744147 (pred)	22.08 (actual)
Steering angle: 5.95849951068139 (pred)	22.08 (actual)
Steering angle: 6.067430355772516 (pred)	21.98 (actual)
Steering angle: 6.676232010365609 (pred)	21.78 (actual)
Steering angle: 7.385883115601233 (pred)	21.68 (actual)
Steering angle: 7.637755732755562 (pred)	21.68 (actual)
Steering angle: 8.711025191948332 (pred)	21.68 (actual)
Steering angle: 9.19156315100458 (pred)	0.0 (actu

...

Steering angle: 3.8294561218548386 (pred)	8.469999999999999 (actual)
Steering angle: 3.727877548518409 (pred)	8.570000000000002 (actual)
Steering angle: 3.665511944980669 (pred)	8.570000000000002 (actual)
Steering angle: 3.864961152707857 (pred)	8.570000000000002 (actual)
Steering angle: 3.539635400558742 (pred)	8.77 (actual)
Steering angle: 4.134018098927115 (pred)	9.08 (actual)
Steering angle: 4.005170422216232 (pred)	9.28 (actual)
Steering angle: 3.957737746638256 (pred)	9.28 (actual)
Steering angle: 4.308533274202349 (pred)	9.28 (actual)
Steering angle: 3.8573672630112723 (pred)	9.380000000000003 (actual)
Steering angle: 4.0857115860221995 (pred)	9.380000000000003 (actual)
Steering angle: 4.2171205836718375 (pred)	9.380000000000003 (actual)
Steering angle: 4.293768112764074 (pred)	9.380000000000003 (actual)
Steering angle: 4.24155729297485 (pred)	9.48 (actual)
Steering angle: 4.48783214321753 (pred)	10.389999999999999 (actual)
Steering angle: 4.399379912805396 (pred)	11.5 (actual)


Steering angle: 8.941019859417464 (pred)	13.610000000000001 (actual)
Steering angle: 8.525280758790775 (pred)	13.71 (actual)
Steering angle: 8.516375045886724 (pred)	13.71 (actual)
Steering angle: 8.825068860520036 (pred)	13.71 (actual)
Steering angle: 9.0570946554936 (pred)	13.41 (actual)
Steering angle: 8.779703597826456 (pred)	13.01 (actual)
Steering angle: 8.53588633502451 (pred)	12.4 (actual)
Steering angle: 8.146414434398398 (pred)	11.699999999999998 (actual)
Steering angle: 8.718481197201047 (pred)	11.19 (actual)
Steering angle: 7.933021395480629 (pred)	11.19 (actual)
Steering angle: 8.381002404084363 (pred)	11.19 (actual)
Steering angle: 8.341262655945936 (pred)	11.19 (actual)
Steering angle: 8.617778595625673 (pred)	11.089999999999998 (actual)
Steering angle: 8.848235154637425 (pred)	11.089999999999998 (actual)
Steering angle: 9.19568090130045 (pred)	11.089999999999998 (actual)
Steering angle: 9.239383012931132 (pred)	10.89 (actual)
Steering angle: 9.249936508972446 (pred)	10.

Steering angle: 7.223954698305864 (pred)	11.8 (actual)
Steering angle: 7.659778823960623 (pred)	12.709999999999999 (actual)
Steering angle: 9.345892129074663 (pred)	13.01 (actual)
Steering angle: 10.063978944822077 (pred)	13.01 (actual)
Steering angle: 10.250646307291232 (pred)	13.01 (actual)
Steering angle: 10.149270078308964 (pred)	13.01 (actual)
Steering angle: 9.89110088051353 (pred)	12.709999999999999 (actual)
Steering angle: 10.18001532108413 (pred)	12.61 (actual)
Steering angle: 9.71169571667939 (pred)	12.4 (actual)
Steering angle: 9.427765606655607 (pred)	11.8 (actual)
Steering angle: 9.487722714737272 (pred)	11.290000000000001 (actual)
Steering angle: 9.68277840328085 (pred)	10.89 (actual)
Steering angle: 10.04795446725558 (pred)	10.79 (actual)
Steering angle: 10.594992001844718 (pred)	10.79 (actual)
Steering angle: 11.161431542544923 (pred)	10.79 (actual)
Steering angle: 11.140055611763723 (pred)	10.79 (actual)
Steering angle: 10.822360461577981 (pred)	10.79 (actual)
Steering

Steering angle: 11.528114517005289 (pred)	21.079999999999995 (actual)
Steering angle: 9.57085209335185 (pred)	21.079999999999995 (actual)
Steering angle: 10.655691892998622 (pred)	20.869999999999997 (actual)
Steering angle: 10.283217771895732 (pred)	20.469999999999995 (actual)
Steering angle: 9.866534397616286 (pred)	19.87 (actual)
Steering angle: 10.144612743068663 (pred)	19.36 (actual)
Steering angle: 10.940271724767202 (pred)	18.66 (actual)
Steering angle: 12.550569930094102 (pred)	17.850000000000005 (actual)
Steering angle: 13.058386810921734 (pred)	17.55 (actual)
Steering angle: 12.728512579672213 (pred)	17.65 (actual)
Steering angle: 12.449383238861303 (pred)	17.55 (actual)
Steering angle: 12.31742739919124 (pred)	17.75 (actual)
Steering angle: 12.240042209668642 (pred)	17.75 (actual)
Steering angle: 11.88555795306592 (pred)	17.75 (actual)
Steering angle: 11.290650183905102 (pred)	17.75 (actual)
Steering angle: 10.700149594306234 (pred)	17.75 (actual)
Steering angle: 11.004509653

IndexError: list index out of range

### Model with Softmax Activation unit

In [10]:
import tensorflow as tf
import scipy

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W, stride):
  return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='VALID')

x = tf.placeholder(tf.float32, shape=[None, 66, 200, 3])
y_ = tf.placeholder(tf.float32, shape=[None, 1])

x_image = x

#first convolutional layer
W_conv1 = weight_variable([5, 5, 3, 24])
b_conv1 = bias_variable([24])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 2) + b_conv1)

#second convolutional layer
W_conv2 = weight_variable([5, 5, 24, 36])
b_conv2 = bias_variable([36])

h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 2) + b_conv2)

#third convolutional layer
W_conv3 = weight_variable([5, 5, 36, 48])
b_conv3 = bias_variable([48])

h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 2) + b_conv3)

#fourth convolutional layer
W_conv4 = weight_variable([3, 3, 48, 64])
b_conv4 = bias_variable([64])

h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4, 1) + b_conv4)

#fifth convolutional layer
W_conv5 = weight_variable([3, 3, 64, 64])
b_conv5 = bias_variable([64])

h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, 1) + b_conv5)

#FCL 1
W_fc1 = weight_variable([1152, 1164])
b_fc1 = bias_variable([1164])

h_conv5_flat = tf.reshape(h_conv5, [-1, 1152])
h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

keep_prob = 0.5                                # dropout of 50%
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#FCL 2
W_fc2 = weight_variable([1164, 100])
b_fc2 = bias_variable([100])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

#FCL 3
W_fc3 = weight_variable([100, 50])
b_fc3 = bias_variable([50])

h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)

#FCL 3
W_fc4 = weight_variable([50, 10])
b_fc4 = bias_variable([10])

h_fc4 = tf.nn.relu(tf.matmul(h_fc3_drop, W_fc4) + b_fc4)

h_fc4_drop = tf.nn.dropout(h_fc4, keep_prob)

#Output
W_fc5 = weight_variable([10, 1])
b_fc5 = bias_variable([1])

# https://keras.io/activations/
# https://www.tensorflow.org/api_docs/python/tf/keras/activations/linear

y = tf.multiply(tf.keras.activations.softmax(tf.matmul(h_fc4_drop, W_fc5) + b_fc5), 2) 


### Training the data
#### Linear activation unit 

In [11]:
import os
import tensorflow as tf
from tensorflow.core.protobuf import saver_pb2
import driving_data
import model

startTime = datetime.datetime.now()
print("Current Time = ",startTime)

LOGDIR = './save_assignment'

sess = tf.InteractiveSession()

L2NormConst = 0.01

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(model.y_, model.y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)
sess.run(tf.initialize_all_variables())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op =  tf.summary.merge_all()

saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V1)

# op to write logs to Tensorboard
logs_path = './logs_assignment'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 30
batch_size = 100

# train over the dataset about 30 times
for epoch in range(epochs):
  for i in range(int(driving_data.num_images/batch_size)):
    xs, ys = driving_data.LoadTrainBatch(batch_size)
    train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.8})
    if i % 10 == 0:
      xs, ys = driving_data.LoadValBatch(batch_size)
      loss_value = loss.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
      print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

    # write logs at every iteration
    summary = merged_summary_op.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
    summary_writer.add_summary(summary, epoch * driving_data.num_images/batch_size + i)

    if i % batch_size == 0:
      if not os.path.exists(LOGDIR):
        os.makedirs(LOGDIR)
      checkpoint_path = os.path.join(LOGDIR, "model_softmax.ckpt")
      filename = saver.save(sess, checkpoint_path)
  print("Model saved in file: %s" % filename)

print("Run the command line:\n" \
          "--> tensorboard --logdir=./logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")

print("Time taken for creation of dataframe is {}".format(datetime.datetime.now() - startTime))


Current Time =  2019-05-12 10:40:31.041605
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch: 0, Step: 0, Loss: 182.49
Epoch: 0, Step: 10, Loss: 154.571
Epoch: 0, Step: 20, Loss: 130.136
Epoch: 0, Step: 30, Loss: 109.4
Epoch: 0, Step: 40, Loss: 91.8664
Epoch: 0, Step: 50, Loss: 77.2588
Epoch: 0, Step: 60, Loss: 65.1385
Epoch: 0, Step: 70, Loss: 55.7746
Epoch: 0, Step: 80, Loss: 47.8838
Epoch: 0, Step: 90, Loss: 39.1057
Epoch: 0, Step: 100, Loss: 32.8808
Epoch: 0, Step: 110, Loss: 27.7838
Epoch: 0, Step: 120, Loss: 23.549
Epoch: 0, Step: 130, Loss: 20.0212
Epoch: 0, Step: 140, Loss: 17.0749
Epoch: 0, Step: 150, Loss: 14.6417
Epoch: 0, Step: 160, Loss: 12.5343
Epoch: 0, Step: 170, Loss: 10.7318
Epoch: 0, Step: 180, Loss: 9.37107
Epoch: 0, Step: 190, Loss: 8.40745
Epoch: 0, Step: 200, Loss: 7.06327
Epoch: 0, Step: 210, Loss: 6.27909
Epoch: 0, Step: 220, Loss: 5.39926
Epoch: 0, Step: 230, Loss: 4.77984
Epoch: 0, Step: 240, Loss: 4.25682
Epoch: 0, Step: 250, Lo

Epoch: 2, Step: 210, Loss: 0.406011
Epoch: 2, Step: 220, Loss: 0.225007
Epoch: 2, Step: 230, Loss: 0.202983
Epoch: 2, Step: 240, Loss: 0.166327
Epoch: 2, Step: 250, Loss: 0.267404
Epoch: 2, Step: 260, Loss: 1.17095
Epoch: 2, Step: 270, Loss: 1.42726
Epoch: 2, Step: 280, Loss: 0.203201
Epoch: 2, Step: 290, Loss: 0.269447
Epoch: 2, Step: 300, Loss: 0.131729
Epoch: 2, Step: 310, Loss: 0.128433
Epoch: 2, Step: 320, Loss: 0.131887
Epoch: 2, Step: 330, Loss: 0.130413
Epoch: 2, Step: 340, Loss: 0.15028
Epoch: 2, Step: 350, Loss: 0.135483
Epoch: 2, Step: 360, Loss: 0.113413
Epoch: 2, Step: 370, Loss: 0.240794
Epoch: 2, Step: 380, Loss: 0.533891
Epoch: 2, Step: 390, Loss: 0.252541
Epoch: 2, Step: 400, Loss: 0.193452
Epoch: 2, Step: 410, Loss: 0.101651
Epoch: 2, Step: 420, Loss: 0.0989737
Epoch: 2, Step: 430, Loss: 0.0946127
Epoch: 2, Step: 440, Loss: 0.0910367
Epoch: 2, Step: 450, Loss: 0.153123
Epoch: 2, Step: 460, Loss: 5.51693
Epoch: 2, Step: 470, Loss: 0.280895
Epoch: 2, Step: 480, Loss: 0.

Epoch: 4, Step: 510, Loss: 0.0249053
Epoch: 4, Step: 520, Loss: 0.0462375
Epoch: 4, Step: 530, Loss: 0.0862497
Epoch: 4, Step: 540, Loss: 0.0243158
Epoch: 4, Step: 550, Loss: 0.0350149
Epoch: 4, Step: 560, Loss: 0.140391
Epoch: 4, Step: 570, Loss: 0.41858
Epoch: 4, Step: 580, Loss: 0.247704
Epoch: 4, Step: 590, Loss: 0.0314378
Epoch: 4, Step: 600, Loss: 0.0261635
Epoch: 4, Step: 610, Loss: 0.0265471
Epoch: 4, Step: 620, Loss: 0.0261349
Epoch: 4, Step: 630, Loss: 0.0257651
Epoch: 4, Step: 640, Loss: 0.491716
Epoch: 4, Step: 650, Loss: 4.9089
Epoch: 4, Step: 660, Loss: 0.0552122
Epoch: 4, Step: 670, Loss: 0.0255734
Epoch: 4, Step: 680, Loss: 0.0219455
Epoch: 4, Step: 690, Loss: 0.0279609
Epoch: 4, Step: 700, Loss: 0.0222206
Epoch: 4, Step: 710, Loss: 0.0197875
Epoch: 4, Step: 720, Loss: 0.0120164
Epoch: 4, Step: 730, Loss: 0.0123901
Epoch: 4, Step: 740, Loss: 0.0120413
Epoch: 4, Step: 750, Loss: 0.0113563
Epoch: 4, Step: 760, Loss: 0.0137249
Epoch: 4, Step: 770, Loss: 0.0481817
Epoch: 4,

Epoch: 6, Step: 810, Loss: 0.0113623
Epoch: 6, Step: 820, Loss: 0.0140146
Epoch: 6, Step: 830, Loss: 1.55659
Epoch: 6, Step: 840, Loss: 4.05821
Epoch: 6, Step: 850, Loss: 0.0162314
Epoch: 6, Step: 860, Loss: 0.00600271
Epoch: 6, Step: 870, Loss: 0.00498281
Epoch: 6, Step: 880, Loss: 0.00792745
Epoch: 6, Step: 890, Loss: 0.00660282
Epoch: 6, Step: 900, Loss: 0.00744261
Epoch: 6, Step: 910, Loss: 0.00680175
Epoch: 6, Step: 920, Loss: 0.00680782
Epoch: 6, Step: 930, Loss: 0.00710459
Epoch: 6, Step: 940, Loss: 0.00343646
Epoch: 6, Step: 950, Loss: 0.00834414
Epoch: 6, Step: 960, Loss: 0.0433162
Epoch: 6, Step: 970, Loss: 0.0423857
Epoch: 6, Step: 980, Loss: 0.0109109
Epoch: 6, Step: 990, Loss: 0.00266148
Epoch: 6, Step: 1000, Loss: 0.00314749
Epoch: 6, Step: 1010, Loss: 0.00340287
Epoch: 6, Step: 1020, Loss: 0.00252269
Epoch: 6, Step: 1030, Loss: 0.00324948
Epoch: 6, Step: 1040, Loss: 0.00289613
Epoch: 6, Step: 1050, Loss: 0.00209599
Model saved in file: ./save_assignment\model_softmax.ckp

Epoch: 8, Step: 1100, Loss: 0.00179493
Epoch: 8, Step: 1110, Loss: 0.00182929
Epoch: 8, Step: 1120, Loss: 0.00267194
Epoch: 8, Step: 1130, Loss: 0.000946573
Epoch: 8, Step: 1140, Loss: 0.0124514
Epoch: 8, Step: 1150, Loss: 0.0528363
Epoch: 8, Step: 1160, Loss: 0.0444241
Epoch: 8, Step: 1170, Loss: 0.00134755
Epoch: 8, Step: 1180, Loss: 0.00122778
Epoch: 8, Step: 1190, Loss: 0.00245484
Epoch: 8, Step: 1200, Loss: 0.00225419
Epoch: 8, Step: 1210, Loss: 0.002266
Epoch: 8, Step: 1220, Loss: 0.00585837
Epoch: 8, Step: 1230, Loss: 0.00109409
Epoch: 8, Step: 1240, Loss: 0.00275239
Epoch: 8, Step: 1250, Loss: 0.0067176
Model saved in file: ./save_assignment\model_softmax.ckpt
Epoch: 9, Step: 900, Loss: 0.0035652
Epoch: 9, Step: 910, Loss: 0.00541002
Epoch: 9, Step: 920, Loss: 0.00296617
Epoch: 9, Step: 930, Loss: 0.00249677
Epoch: 9, Step: 940, Loss: 0.00282791
Epoch: 9, Step: 950, Loss: 0.00378957
Epoch: 9, Step: 960, Loss: 0.0386872
Epoch: 9, Step: 970, Loss: 0.0330856
Epoch: 9, Step: 980, L

...

Epoch: 21, Step: 2210, Loss: 0.0480855
Epoch: 21, Step: 2220, Loss: 0.0146196
Epoch: 21, Step: 2230, Loss: 0.0129787
Epoch: 21, Step: 2240, Loss: 0.0316832
Epoch: 21, Step: 2250, Loss: 0.151634
Epoch: 21, Step: 2260, Loss: 0.0408506
Epoch: 21, Step: 2270, Loss: 0.120741
Epoch: 21, Step: 2280, Loss: 0.0548539
Epoch: 21, Step: 2290, Loss: 0.0401711
Epoch: 21, Step: 2300, Loss: 0.0257164
Epoch: 21, Step: 2310, Loss: 0.0213038
Epoch: 21, Step: 2320, Loss: 0.0222505
Epoch: 21, Step: 2330, Loss: 0.0231237
Epoch: 21, Step: 2340, Loss: 0.0234761
Epoch: 21, Step: 2350, Loss: 0.0228705
Epoch: 21, Step: 2360, Loss: 0.0272307
Epoch: 21, Step: 2370, Loss: 0.0330705
Epoch: 21, Step: 2380, Loss: 0.040876
Epoch: 21, Step: 2390, Loss: 0.00868773
Epoch: 21, Step: 2400, Loss: 0.0194873
Epoch: 21, Step: 2410, Loss: 0.0822695
Epoch: 21, Step: 2420, Loss: 0.0237175
Epoch: 21, Step: 2430, Loss: 0.321292
Epoch: 21, Step: 2440, Loss: 0.604634
Epoch: 21, Step: 2450, Loss: 0.229496
Epoch: 21, Step: 2460, Loss: 0

Epoch: 23, Step: 2490, Loss: 0.0154089
Epoch: 23, Step: 2500, Loss: 0.0127961
Epoch: 23, Step: 2510, Loss: 0.0149266
Epoch: 23, Step: 2520, Loss: 0.0154105
Epoch: 23, Step: 2530, Loss: 0.0148803
Epoch: 23, Step: 2540, Loss: 0.0154985
Epoch: 23, Step: 2550, Loss: 0.0278543
Epoch: 23, Step: 2560, Loss: 0.0240715
Epoch: 23, Step: 2570, Loss: 0.0339626
Epoch: 23, Step: 2580, Loss: 0.00486926
Epoch: 23, Step: 2590, Loss: 0.0290989
Epoch: 23, Step: 2600, Loss: 0.0946353
Epoch: 23, Step: 2610, Loss: 0.0146074
Epoch: 23, Step: 2620, Loss: 0.452534
Epoch: 23, Step: 2630, Loss: 0.615824
Epoch: 23, Step: 2640, Loss: 0.208681
Epoch: 23, Step: 2650, Loss: 0.0490928
Epoch: 23, Step: 2660, Loss: 0.0400583
Epoch: 23, Step: 2670, Loss: 0.0148041
Epoch: 23, Step: 2680, Loss: 0.141519
Epoch: 23, Step: 2690, Loss: 1.14777
Epoch: 23, Step: 2700, Loss: 1.16891
Epoch: 23, Step: 2710, Loss: 0.0534132
Epoch: 23, Step: 2720, Loss: 0.111289
Epoch: 23, Step: 2730, Loss: 0.00246474
Epoch: 23, Step: 2740, Loss: 0.0

Epoch: 25, Step: 2710, Loss: 0.0223516
Epoch: 25, Step: 2720, Loss: 0.0223488
Epoch: 25, Step: 2730, Loss: 0.0228965
Epoch: 25, Step: 2740, Loss: 0.0385005
Epoch: 25, Step: 2750, Loss: 0.0307402
Epoch: 25, Step: 2760, Loss: 0.0337126
Epoch: 25, Step: 2770, Loss: 0.00355395
Epoch: 25, Step: 2780, Loss: 0.0436123
Epoch: 25, Step: 2790, Loss: 0.0914209
Epoch: 25, Step: 2800, Loss: 0.0187837
Epoch: 25, Step: 2810, Loss: 0.538205
Epoch: 25, Step: 2820, Loss: 0.536811
Epoch: 25, Step: 2830, Loss: 0.148669
Epoch: 25, Step: 2840, Loss: 0.0337423
Epoch: 25, Step: 2850, Loss: 0.0214005
Epoch: 25, Step: 2860, Loss: 0.041419
Epoch: 25, Step: 2870, Loss: 0.293517
Epoch: 25, Step: 2880, Loss: 1.44679
Epoch: 25, Step: 2890, Loss: 0.898787
Epoch: 25, Step: 2900, Loss: 0.0659156
Epoch: 25, Step: 2910, Loss: 0.0541762
Epoch: 25, Step: 2920, Loss: 0.0043444
Epoch: 25, Step: 2930, Loss: 0.00388889
Epoch: 25, Step: 2940, Loss: 0.00947599
Epoch: 25, Step: 2950, Loss: 0.0258999
Model saved in file: ./save_as

Epoch: 27, Step: 3000, Loss: 0.602648
Epoch: 27, Step: 3010, Loss: 0.473064
Epoch: 27, Step: 3020, Loss: 0.134063
Epoch: 27, Step: 3030, Loss: 0.0446786
Epoch: 27, Step: 3040, Loss: 0.0193016
Epoch: 27, Step: 3050, Loss: 0.0559648
Epoch: 27, Step: 3060, Loss: 0.441087
Epoch: 27, Step: 3070, Loss: 1.40236
Epoch: 27, Step: 3080, Loss: 0.59572
Epoch: 27, Step: 3090, Loss: 0.112413
Epoch: 27, Step: 3100, Loss: 0.0238552
Epoch: 27, Step: 3110, Loss: 0.00561549
Epoch: 27, Step: 3120, Loss: 0.000740292
Epoch: 27, Step: 3130, Loss: 0.0101212
Epoch: 27, Step: 3140, Loss: 0.0271955
Epoch: 27, Step: 3150, Loss: 0.0506015
Model saved in file: ./save_assignment\model_softmax.ckpt
Epoch: 28, Step: 2800, Loss: 0.00208398
Epoch: 28, Step: 2810, Loss: 0.0755443
Epoch: 28, Step: 2820, Loss: 0.154015
Epoch: 28, Step: 2830, Loss: 0.340028
Epoch: 28, Step: 2840, Loss: 0.208168
Epoch: 28, Step: 2850, Loss: 0.0114706
Epoch: 28, Step: 2860, Loss: 0.0083797
Epoch: 28, Step: 2870, Loss: 0.00850466
Epoch: 28, St

Epoch: 29, Step: 3210, Loss: 0.108048
Epoch: 29, Step: 3220, Loss: 0.0439471
Epoch: 29, Step: 3230, Loss: 0.012054
Epoch: 29, Step: 3240, Loss: 0.0958465
Epoch: 29, Step: 3250, Loss: 0.702732
Epoch: 29, Step: 3260, Loss: 1.51073
Epoch: 29, Step: 3270, Loss: 0.379467
Epoch: 29, Step: 3280, Loss: 0.10759
Epoch: 29, Step: 3290, Loss: 0.00458638
Epoch: 29, Step: 3300, Loss: 0.00731326
Epoch: 29, Step: 3310, Loss: 0.00346005
Epoch: 29, Step: 3320, Loss: 0.0123221
Epoch: 29, Step: 3330, Loss: 0.0400203
Epoch: 29, Step: 3340, Loss: 0.0484871
Epoch: 29, Step: 3350, Loss: 0.00542248
Model saved in file: ./save_assignment\model_softmax.ckpt
Run the command line:
--> tensorboard --logdir=./logs 
Then open http://0.0.0.0:6006/ into your web browser
Time taken for creation of dataframe is 22:00:05.370022


## Running dataset

In [12]:
#pip3 install opencv-python

import tensorflow as tf
import scipy.misc
import model
import cv2
from subprocess import call
import math

sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, "save_assignment/model_softmax.ckpt")

img = cv2.imread('steering_wheel_image.jpg',0)
rows,cols = img.shape

smoothed_angle = 0

#read data.txt
xs = []
ys = []
with open("driving_dataset/data.txt") as f:
    for line in f:
        xs.append("driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)


i = math.ceil(num_images*0.7)
print("Starting frameofvideo:" +str(i))

while(cv2.waitKey(10) != ord('q')):
    full_image = scipy.misc.imread("driving_dataset/" + str(i) + ".jpg", mode="RGB")
    image = scipy.misc.imresize(full_image[-150:], [66, 200]) / 255.0
    degrees = model.y.eval(feed_dict={model.x: [image], model.keep_prob: 1.0})[0][0] * 180.0 / scipy.pi
    #call("clear")
    #print("Predicted Steering angle: " + str(degrees))
    print("Steering angle: " + str(degrees) + " (pred)\t" + str(ys[i]*180/scipy.pi) + " (actual)")
    cv2.imshow("frame", cv2.cvtColor(full_image, cv2.COLOR_RGB2BGR))
    #make smooth angle transitions by turning the steering wheel based on the difference of the current angle
    #and the predicted angle
    smoothed_angle += 0.3 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    cv2.imshow("steering wheel", dst)
    i += 1

cv2.destroyAllWindows()


INFO:tensorflow:Restoring parameters from save_assignment/model_softmax.ckpt
Starting frameofvideo:31785


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


Steering angle: 2.606208088206459 (pred)	-28.34 (actual)
Steering angle: 2.606208088206459 (pred)	-28.840000000000003 (actual)
Steering angle: 2.606208088206459 (pred)	-29.75 (actual)
Steering angle: 2.606208088206459 (pred)	-31.06 (actual)
Steering angle: 2.606208088206459 (pred)	-32.27 (actual)
Steering angle: 2.606208088206459 (pred)	-33.48 (actual)
Steering angle: 2.606208088206459 (pred)	-34.39 (actual)
Steering angle: 2.606208088206459 (pred)	-35.599999999999994 (actual)
Steering angle: 2.606208088206459 (pred)	-36.5 (actual)
Steering angle: 2.606208088206459 (pred)	-37.61 (actual)
Steering angle: 2.606208088206459 (pred)	-38.62 (actual)
Steering angle: 2.606208088206459 (pred)	-39.63 (actual)
Steering angle: 2.606208088206459 (pred)	-39.93000000000001 (actual)
Steering angle: 2.606208088206459 (pred)	-40.03000000000001 (actual)
Steering angle: 2.606208088206459 (pred)	-40.03000000000001 (actual)
Steering angle: 2.606208088206459 (pred)	-40.03000000000001 (actual)
Steering angle:

Steering angle: 2.606208088206459 (pred)	-0.61 (actual)
Steering angle: 2.606208088206459 (pred)	-0.61 (actual)
Steering angle: 2.606208088206459 (pred)	-0.61 (actual)
Steering angle: 2.606208088206459 (pred)	-0.61 (actual)
Steering angle: 2.606208088206459 (pred)	-0.61 (actual)
Steering angle: 2.606208088206459 (pred)	-0.5 (actual)
Steering angle: 2.606208088206459 (pred)	-0.3 (actual)
Steering angle: 2.606208088206459 (pred)	0.2 (actual)
Steering angle: 2.606208088206459 (pred)	0.3 (actual)
Steering angle: 2.606208088206459 (pred)	0.4 (actual)
Steering angle: 2.606208088206459 (pred)	0.5 (actual)
Steering angle: 2.606208088206459 (pred)	1.01 (actual)
Steering angle: 2.606208088206459 (pred)	1.6100000000000003 (actual)
Steering angle: 2.606208088206459 (pred)	2.12 (actual)
Steering angle: 2.606208088206459 (pred)	2.52 (actual)
Steering angle: 2.606208088206459 (pred)	3.13 (actual)
Steering angle: 2.606208088206459 (pred)	4.13 (actual)
Steering angle: 2.606208088206459 (pred)	4.64 (act

Steering angle: 2.606208088206459 (pred)	23.39 (actual)
Steering angle: 2.606208088206459 (pred)	23.29 (actual)
Steering angle: 2.606208088206459 (pred)	22.89 (actual)
Steering angle: 2.606208088206459 (pred)	22.59 (actual)
Steering angle: 2.606208088206459 (pred)	22.59 (actual)
Steering angle: 2.606208088206459 (pred)	22.39 (actual)
Steering angle: 2.606208088206459 (pred)	22.08 (actual)
Steering angle: 2.606208088206459 (pred)	22.08 (actual)
Steering angle: 2.606208088206459 (pred)	22.08 (actual)
Steering angle: 2.606208088206459 (pred)	21.98 (actual)
Steering angle: 2.606208088206459 (pred)	21.78 (actual)
Steering angle: 2.606208088206459 (pred)	21.68 (actual)
Steering angle: 2.606208088206459 (pred)	21.68 (actual)
Steering angle: 2.606208088206459 (pred)	21.68 (actual)
Steering angle: 2.606208088206459 (pred)	0.0 (actual)
Steering angle: 2.606208088206459 (pred)	22.69 (actual)
Steering angle: 2.606208088206459 (pred)	23.189999999999998 (actual)
Steering angle: 2.606208088206459 (pr

Steering angle: 2.606208088206459 (pred)	39.13 (actual)
Steering angle: 2.606208088206459 (pred)	39.43 (actual)
Steering angle: 2.606208088206459 (pred)	39.830000000000005 (actual)
Steering angle: 2.606208088206459 (pred)	39.93000000000001 (actual)
Steering angle: 2.606208088206459 (pred)	39.93000000000001 (actual)
Steering angle: 2.606208088206459 (pred)	40.13 (actual)
Steering angle: 2.606208088206459 (pred)	40.64000000000001 (actual)
Steering angle: 2.606208088206459 (pred)	41.24 (actual)
Steering angle: 2.606208088206459 (pred)	41.75 (actual)
Steering angle: 2.606208088206459 (pred)	42.959999999999994 (actual)
Steering angle: 2.606208088206459 (pred)	43.97 (actual)
Steering angle: 2.606208088206459 (pred)	44.57 (actual)
Steering angle: 2.606208088206459 (pred)	45.18 (actual)
Steering angle: 2.606208088206459 (pred)	45.88 (actual)
Steering angle: 2.606208088206459 (pred)	46.39 (actual)
Steering angle: 2.606208088206459 (pred)	46.69 (actual)
Steering angle: 2.606208088206459 (pred)	4

Steering angle: 2.606208088206459 (pred)	0.0 (actual)
Steering angle: 2.606208088206459 (pred)	3.13 (actual)
Steering angle: 2.606208088206459 (pred)	2.92 (actual)
Steering angle: 2.606208088206459 (pred)	2.62 (actual)
Steering angle: 2.606208088206459 (pred)	1.92 (actual)
Steering angle: 2.606208088206459 (pred)	1.21 (actual)
Steering angle: 2.606208088206459 (pred)	1.01 (actual)


IndexError: too many indices for array

### Model with tanh Activation unit

In [9]:
import tensorflow as tf
import scipy

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W, stride):
  return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='VALID')

x = tf.placeholder(tf.float32, shape=[None, 66, 200, 3])
y_ = tf.placeholder(tf.float32, shape=[None, 1])

x_image = x

#first convolutional layer
W_conv1 = weight_variable([5, 5, 3, 24])
b_conv1 = bias_variable([24])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 2) + b_conv1)

#second convolutional layer
W_conv2 = weight_variable([5, 5, 24, 36])
b_conv2 = bias_variable([36])

h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 2) + b_conv2)

#third convolutional layer
W_conv3 = weight_variable([5, 5, 36, 48])
b_conv3 = bias_variable([48])

h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 2) + b_conv3)

#fourth convolutional layer
W_conv4 = weight_variable([3, 3, 48, 64])
b_conv4 = bias_variable([64])

h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4, 1) + b_conv4)

#fifth convolutional layer
W_conv5 = weight_variable([3, 3, 64, 64])
b_conv5 = bias_variable([64])

h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, 1) + b_conv5)

#FCL 1
W_fc1 = weight_variable([1152, 1164])
b_fc1 = bias_variable([1164])

h_conv5_flat = tf.reshape(h_conv5, [-1, 1152])
h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

keep_prob = 0.5                                # dropout of 50%
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#FCL 2
W_fc2 = weight_variable([1164, 100])
b_fc2 = bias_variable([100])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

#FCL 3
W_fc3 = weight_variable([100, 50])
b_fc3 = bias_variable([50])

h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)

#FCL 3
W_fc4 = weight_variable([50, 10])
b_fc4 = bias_variable([10])

h_fc4 = tf.nn.relu(tf.matmul(h_fc3_drop, W_fc4) + b_fc4)

h_fc4_drop = tf.nn.dropout(h_fc4, keep_prob)

#Output
W_fc5 = weight_variable([10, 1])
b_fc5 = bias_variable([1])

# https://keras.io/activations/
# https://www.tensorflow.org/api_docs/python/tf/keras/activations/linear

y = tf.multiply(tf.keras.activations.tanh(tf.matmul(h_fc4_drop, W_fc5) + b_fc5), 2) 


### Training the data
#### tanh activation unit 

In [10]:
import os
import tensorflow as tf
from tensorflow.core.protobuf import saver_pb2
import driving_data
import model

startTime = datetime.datetime.now()
print("Current Time = ",startTime)

LOGDIR = './save_assignment'

sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(model.y_, model.y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)
sess.run(tf.initialize_all_variables())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op =  tf.summary.merge_all()

saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V1)

# op to write logs to Tensorboard
logs_path = './logs_assignment'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 7
batch_size = 100

# train over the dataset about 30 times
for epoch in range(epochs):
  for i in range(int(driving_data.num_images/batch_size)):
    xs, ys = driving_data.LoadTrainBatch(batch_size)
    train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.8})
    if i % 10 == 0:
      xs, ys = driving_data.LoadValBatch(batch_size)
      loss_value = loss.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
      print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

    # write logs at every iteration
    summary = merged_summary_op.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
    summary_writer.add_summary(summary, epoch * driving_data.num_images/batch_size + i)

    if i % batch_size == 0:
      if not os.path.exists(LOGDIR):
        os.makedirs(LOGDIR)
      checkpoint_path = os.path.join(LOGDIR, "model_tanh.ckpt")
      filename = saver.save(sess, checkpoint_path)
  print("Model saved in file: %s" % filename)

print("Run the command line:\n" \
          "--> tensorboard --logdir=./logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")

print("Time taken for creation of dataframe is {}".format(datetime.datetime.now() - startTime))


Current Time =  2019-05-17 17:59:35.330698
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch: 0, Step: 0, Loss: 12.7909
Epoch: 0, Step: 10, Loss: 12.6762
Epoch: 0, Step: 20, Loss: 12.0629
Epoch: 0, Step: 30, Loss: 11.8731
Epoch: 0, Step: 40, Loss: 11.688
Epoch: 0, Step: 50, Loss: 11.5933
Epoch: 0, Step: 60, Loss: 11.5857
Epoch: 0, Step: 70, Loss: 12.2638
Epoch: 0, Step: 80, Loss: 12.8069
Epoch: 0, Step: 90, Loss: 11.1906
Epoch: 0, Step: 100, Loss: 10.8154
Epoch: 0, Step: 110, Loss: 10.6282
Epoch: 0, Step: 120, Loss: 10.4841
Epoch: 0, Step: 130, Loss: 10.3589
Epoch: 0, Step: 140, Loss: 10.2466
Epoch: 0, Step: 150, Loss: 10.1604
Epoch: 0, Step: 160, Loss: 10.0025
Epoch: 0, Step: 170, Loss: 9.81738
Epoch: 0, Step: 180, Loss: 9.79067
Epoch: 0, Step: 190, Loss: 9.9262
Epoch: 0, Step: 200, Loss: 9.48257
Epoch: 0, Step: 210, Loss: 9.43215
Epoch: 0, Step: 220, Loss: 9.18727
Epoch: 0, Step: 230, Loss: 9.0695
Epoch: 0, Step: 240, Loss: 8.95342
Epoch: 0, Step: 250, L

Epoch: 2, Step: 220, Loss: 4.20573
Epoch: 2, Step: 230, Loss: 4.15673
Epoch: 2, Step: 240, Loss: 4.13684
Epoch: 2, Step: 250, Loss: 4.23882
Epoch: 2, Step: 260, Loss: 5.17491
Epoch: 2, Step: 270, Loss: 5.37079
Epoch: 2, Step: 280, Loss: 4.02037
Epoch: 2, Step: 290, Loss: 4.00454
Epoch: 2, Step: 300, Loss: 3.88435
Epoch: 2, Step: 310, Loss: 3.87196
Epoch: 2, Step: 320, Loss: 3.82618
Epoch: 2, Step: 330, Loss: 3.79798
Epoch: 2, Step: 340, Loss: 3.78586
Epoch: 2, Step: 350, Loss: 3.75454
Epoch: 2, Step: 360, Loss: 3.69357
Epoch: 2, Step: 370, Loss: 3.74649
Epoch: 2, Step: 380, Loss: 4.02972
Epoch: 2, Step: 390, Loss: 3.8773
Epoch: 2, Step: 400, Loss: 3.62652
Epoch: 2, Step: 410, Loss: 3.52418
Epoch: 2, Step: 420, Loss: 3.49581
Epoch: 2, Step: 430, Loss: 3.46584
Epoch: 2, Step: 440, Loss: 3.4373
Epoch: 2, Step: 450, Loss: 3.483
Epoch: 2, Step: 460, Loss: 9.15255
Epoch: 2, Step: 470, Loss: 3.603
Epoch: 2, Step: 480, Loss: 3.33707
Epoch: 2, Step: 490, Loss: 3.30286
Epoch: 2, Step: 500, Loss:

Epoch: 4, Step: 540, Loss: 2.03797
Epoch: 4, Step: 550, Loss: 2.04327
Epoch: 4, Step: 560, Loss: 2.12875
Epoch: 4, Step: 570, Loss: 2.39842
Epoch: 4, Step: 580, Loss: 2.25793
Epoch: 4, Step: 590, Loss: 1.98676
Epoch: 4, Step: 600, Loss: 1.96901
Epoch: 4, Step: 610, Loss: 1.95944
Epoch: 4, Step: 620, Loss: 1.94955
Epoch: 4, Step: 630, Loss: 1.94112
Epoch: 4, Step: 640, Loss: 2.42601
Epoch: 4, Step: 650, Loss: 7.13663
Epoch: 4, Step: 660, Loss: 1.9619
Epoch: 4, Step: 670, Loss: 1.90703
Epoch: 4, Step: 680, Loss: 1.89579
Epoch: 4, Step: 690, Loss: 1.88763
Epoch: 4, Step: 700, Loss: 1.89564
Epoch: 4, Step: 710, Loss: 1.88669
Epoch: 4, Step: 720, Loss: 1.90291
Epoch: 4, Step: 730, Loss: 1.86033
Epoch: 4, Step: 740, Loss: 1.886
Epoch: 4, Step: 750, Loss: 1.88598
Epoch: 4, Step: 760, Loss: 2.01586
Epoch: 4, Step: 770, Loss: 1.84161
Epoch: 4, Step: 780, Loss: 1.87888
Epoch: 4, Step: 790, Loss: 1.89595
Epoch: 4, Step: 800, Loss: 1.82318
Epoch: 4, Step: 810, Loss: 1.86848
Epoch: 4, Step: 820, Lo

Epoch: 6, Step: 860, Loss: 1.35679
Epoch: 6, Step: 870, Loss: 1.34493
Epoch: 6, Step: 880, Loss: 1.32593
Epoch: 6, Step: 890, Loss: 1.32873
Epoch: 6, Step: 900, Loss: 1.31776
Epoch: 6, Step: 910, Loss: 1.32761
Epoch: 6, Step: 920, Loss: 1.31996
Epoch: 6, Step: 930, Loss: 1.32152
Epoch: 6, Step: 940, Loss: 1.30022
Epoch: 6, Step: 950, Loss: 1.29969
Epoch: 6, Step: 960, Loss: 1.29309
Epoch: 6, Step: 970, Loss: 1.2817
Epoch: 6, Step: 980, Loss: 1.28765
Epoch: 6, Step: 990, Loss: 1.27105
Epoch: 6, Step: 1000, Loss: 1.27132
Epoch: 6, Step: 1010, Loss: 1.2618
Epoch: 6, Step: 1020, Loss: 1.26321
Epoch: 6, Step: 1030, Loss: 1.25489
Epoch: 6, Step: 1040, Loss: 1.25518
Epoch: 6, Step: 1050, Loss: 1.28054
Model saved in file: ./save_assignment\model_tanh.ckpt
Run the command line:
--> tensorboard --logdir=./logs 
Then open http://0.0.0.0:6006/ into your web browser
Time taken for creation of dataframe is 4:28:21.957420


## Running dataset

In [11]:
#pip3 install opencv-python

import tensorflow as tf
import scipy.misc
import model
import cv2
from subprocess import call
import math

sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, "save_assignment/model_tanh.ckpt")

img = cv2.imread('steering_wheel_image.jpg',0)
rows,cols = img.shape

smoothed_angle = 0

#read data.txt
xs = []
ys = []
with open("driving_dataset/data.txt") as f:
    for line in f:
        xs.append("driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)


i = math.ceil(num_images*0.7)
print("Starting frameofvideo:" +str(i))

while(cv2.waitKey(10) != ord('q')):
    full_image = scipy.misc.imread("driving_dataset/" + str(i) + ".jpg", mode="RGB")
    image = scipy.misc.imresize(full_image[-150:], [66, 200]) / 255.0
    degrees = model.y.eval(feed_dict={model.x: [image], model.keep_prob: 1.0})[0][0] * 180.0 / scipy.pi
    #call("clear")
    #print("Predicted Steering angle: " + str(degrees))
    print("Steering angle: " + str(degrees) + " (pred)\t" + str(ys[i]*180/scipy.pi) + " (actual)")
    cv2.imshow("frame", cv2.cvtColor(full_image, cv2.COLOR_RGB2BGR))
    #make smooth angle transitions by turning the steering wheel based on the difference of the current angle
    #and the predicted angle
    smoothed_angle += 0.3 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    cv2.imshow("steering wheel", dst)
    i += 1

cv2.destroyAllWindows()


INFO:tensorflow:Restoring parameters from save_assignment/model_tanh.ckpt
Starting frameofvideo:31785


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


Steering angle: 3.923803232407517 (pred)	-28.34 (actual)
Steering angle: 4.497834955257014 (pred)	-28.840000000000003 (actual)
Steering angle: 4.472712238829238 (pred)	-29.75 (actual)
Steering angle: 5.932216088981535 (pred)	-31.06 (actual)
Steering angle: 6.136848556515117 (pred)	-32.27 (actual)
Steering angle: 6.864962516850932 (pred)	-33.48 (actual)
Steering angle: 6.355220376451003 (pred)	-34.39 (actual)
Steering angle: 5.902768155261849 (pred)	-35.599999999999994 (actual)
Steering angle: 6.791526884215894 (pred)	-36.5 (actual)
Steering angle: 6.27035186091901 (pred)	-37.61 (actual)
Steering angle: 6.15290120861194 (pred)	-38.62 (actual)
Steering angle: 6.145928866129826 (pred)	-39.63 (actual)
Steering angle: 5.601656704380898 (pred)	-39.93000000000001 (actual)
Steering angle: 4.744607128648636 (pred)	-40.03000000000001 (actual)
Steering angle: 5.071041505877409 (pred)	-40.03000000000001 (actual)
Steering angle: 5.495822069417904 (pred)	-40.03000000000001 (actual)
Steering angle: 5

Steering angle: 5.353125757277997 (pred)	-0.61 (actual)
Steering angle: 6.02283434313421 (pred)	-0.61 (actual)
Steering angle: 7.00563367790193 (pred)	-0.5 (actual)
Steering angle: 6.889277990296177 (pred)	-0.3 (actual)
Steering angle: 6.612736437407055 (pred)	0.2 (actual)
Steering angle: 7.382080407780829 (pred)	0.3 (actual)
Steering angle: 7.644615804003194 (pred)	0.4 (actual)
Steering angle: 7.155755686330109 (pred)	0.5 (actual)
Steering angle: 6.761289636760248 (pred)	1.01 (actual)
Steering angle: 6.451045796072166 (pred)	1.6100000000000003 (actual)
Steering angle: 6.3351434622725735 (pred)	2.12 (actual)
Steering angle: 6.81996266927795 (pred)	2.52 (actual)
Steering angle: 6.975783616794859 (pred)	3.13 (actual)
Steering angle: 7.196356465190781 (pred)	4.13 (actual)
Steering angle: 7.2644790644251955 (pred)	4.64 (actual)
Steering angle: 7.124746627498252 (pred)	4.94 (actual)
Steering angle: 6.772022425267276 (pred)	4.94 (actual)
Steering angle: 6.053430602408105 (pred)	4.94 (actual)

Steering angle: 7.721211252903009 (pred)	22.08 (actual)
Steering angle: 7.396688474868194 (pred)	22.08 (actual)
Steering angle: 7.364776977292248 (pred)	22.08 (actual)
Steering angle: 7.403331687609677 (pred)	21.98 (actual)
Steering angle: 7.187233040006936 (pred)	21.78 (actual)
Steering angle: 7.466250536470189 (pred)	21.68 (actual)
Steering angle: 7.033930297859646 (pred)	21.68 (actual)
Steering angle: 6.897466960224199 (pred)	21.68 (actual)
Steering angle: 7.086260645959345 (pred)	0.0 (actual)
Steering angle: 7.38815244595297 (pred)	22.69 (actual)
Steering angle: 8.490186393061602 (pred)	23.189999999999998 (actual)
Steering angle: 8.833589521509627 (pred)	23.39 (actual)
Steering angle: 8.319958442151021 (pred)	23.6 (actual)
Steering angle: 8.305477587336947 (pred)	24.1 (actual)
Steering angle: 10.177559014303865 (pred)	24.5 (actual)
Steering angle: 9.453526518883956 (pred)	24.810000000000002 (actual)
Steering angle: 8.19956611274579 (pred)	24.91 (actual)
Steering angle: 8.5785374512

...

Steering angle: 2.387899874407219 (pred)	-44.37 (actual)
Steering angle: 1.842119094625364 (pred)	-44.37 (actual)
Steering angle: 1.1975873331017588 (pred)	-44.47 (actual)
Steering angle: 1.2124755446667999 (pred)	-44.57 (actual)
Steering angle: 1.314541195868416 (pred)	-44.57 (actual)
Steering angle: 1.2811697453573798 (pred)	-44.57 (actual)
Steering angle: 2.2429671022009567 (pred)	-44.57 (actual)
Steering angle: 2.1420162659379334 (pred)	-43.97 (actual)
Steering angle: 3.285639021459362 (pred)	-43.16 (actual)
Steering angle: 3.4689143412319603 (pred)	-42.15 (actual)
Steering angle: 4.165772929038974 (pred)	-40.93999999999999 (actual)
Steering angle: 3.6324495605295755 (pred)	-39.830000000000005 (actual)
Steering angle: 3.22058275027546 (pred)	-38.52 (actual)
Steering angle: 2.6350472813163686 (pred)	-37.61 (actual)
Steering angle: 2.041400962717887 (pred)	-36.90999999999999 (actual)
Steering angle: 2.9333607025151416 (pred)	-35.599999999999994 (actual)
Steering angle: 2.541542205022

Steering angle: 4.361165004399175 (pred)	2.42 (actual)
Steering angle: 4.524878235502033 (pred)	2.32 (actual)
Steering angle: 4.5649210732848795 (pred)	1.41 (actual)
Steering angle: 4.151285670922554 (pred)	0.81 (actual)
Steering angle: 4.1440174958720135 (pred)	0.3 (actual)
Steering angle: 3.8144830665337013 (pred)	-0.4 (actual)
Steering angle: 3.5963622560498134 (pred)	-0.81 (actual)
Steering angle: 3.585185505246734 (pred)	-0.81 (actual)
Steering angle: 3.8763129271084185 (pred)	-0.81 (actual)
Steering angle: 3.581450245544383 (pred)	-1.11 (actual)
Steering angle: 3.308778848593719 (pred)	-2.32 (actual)
Steering angle: 2.9261680864322948 (pred)	-3.13 (actual)
Steering angle: 3.2326301367543047 (pred)	-3.23 (actual)
Steering angle: 3.354228634878276 (pred)	-3.23 (actual)
Steering angle: 3.5529367670805185 (pred)	-3.23 (actual)
Steering angle: 3.5364405795744682 (pred)	-3.23 (actual)
Steering angle: 3.2806642958660657 (pred)	-3.3299999999999996 (actual)
Steering angle: 3.1746463130125

Steering angle: 3.026332383723709 (pred)	-7.87 (actual)
Steering angle: 3.0462679983636836 (pred)	-7.87 (actual)
Steering angle: 3.298787775941871 (pred)	-7.87 (actual)
Steering angle: 3.1422961892258314 (pred)	-7.87 (actual)
Steering angle: 2.8796655972419094 (pred)	-7.87 (actual)
Steering angle: 2.7526360315107197 (pred)	-7.87 (actual)
Steering angle: 2.3922769584481394 (pred)	-7.87 (actual)
Steering angle: 2.2260636646656446 (pred)	-7.87 (actual)
Steering angle: 2.599577682069669 (pred)	-7.87 (actual)
Steering angle: 1.382633806303506 (pred)	-7.87 (actual)
Steering angle: 1.6497148001402204 (pred)	-7.87 (actual)
Steering angle: 1.5961871412233595 (pred)	-7.460000000000001 (actual)
Steering angle: 1.779658188604361 (pred)	-6.860000000000001 (actual)
Steering angle: 1.634847719472924 (pred)	-6.659999999999999 (actual)
Steering angle: 2.1523373221040574 (pred)	0.0 (actual)
Steering angle: 2.0867331416826733 (pred)	-6.45 (actual)
Steering angle: 2.170878937823115 (pred)	-6.45 (actual)
S

Steering angle: 5.7389225892438205 (pred)	-7.87 (actual)
Steering angle: 6.136574495174671 (pred)	-7.87 (actual)
Steering angle: 6.196052209353964 (pred)	-7.87 (actual)
Steering angle: 5.886142621048391 (pred)	-7.87 (actual)
Steering angle: 4.875282307849321 (pred)	-8.27 (actual)
Steering angle: 5.252796681672857 (pred)	-8.67 (actual)
Steering angle: 5.2890893187145425 (pred)	-9.08 (actual)
Steering angle: 4.0561898008821125 (pred)	-9.78 (actual)
Steering angle: 3.5974832608473415 (pred)	-10.29 (actual)
Steering angle: 4.317604619193772 (pred)	-10.389999999999999 (actual)
Steering angle: 4.503969745792155 (pred)	-10.49 (actual)
Steering angle: 3.4909720102696933 (pred)	-10.49 (actual)
Steering angle: 3.295885799318262 (pred)	-10.49 (actual)
Steering angle: 3.7165125406263306 (pred)	-10.49 (actual)
Steering angle: 4.225675395607278 (pred)	-10.59 (actual)
Steering angle: 4.155519534434315 (pred)	-10.99 (actual)
Steering angle: 4.073596111095067 (pred)	-11.6 (actual)
Steering angle: 4.081

KeyboardInterrupt: 